In [1]:
import sys
sys.path.append("../")

In [2]:
import scipy
from post_processing.src import hdf5_tools
from post_processing.src import plotting
from post_processing.src import filter_joints
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from platform import python_version

%matplotlib inline 

In [3]:
python_version()

'3.8.10'

In [4]:
hdf5_files = hdf5_tools.load_hdf5_files("/media/mjsobrep/43CDA61E672B9161/pose")

In [5]:
hdf5_tools.check_data(hdf5_files)

TypeError: tuple indices must be integers or slices, not dict

In [ ]:
all_subj=list(hdf5_files.keys())
all_subj.remove('060')
all_subj.remove('500-3')

In [ ]:
files_list=[hdf5_files[subj] for subj in all_subj]

In [ ]:
with multiprocessing.Pool() as pool:
    res_all = pool.imap(filter_joints.filter_file, files_list)

In [ ]:
next(res_all)

In [ ]:
res=filter_joints.filter_file(hdf5_files['031'])

In [ ]:
gd=res[0]['state']
for joint in ['RWrist', 'LWrist','RElbow','LElbow','RShoulder','LShoulder']:
    plotting.plot_model_results(gd['time'], gd['raw'][joint], gd['smooth'][joint], title=joint)


In [ ]:
# 1) subtract shoulder location from values (would be better to come at this from )
# 2) add shoulder location (0,0) to list to make convex hull
# 3) divide by arm length ^ 3
ch=scipy.spatial.ConvexHull(gd['smooth']['RWrist'][:,0:3]-gd['smooth']['RShoulder'][:,0:3])
# TODO: include the angular effects of shoulder positions 
# Could add in shoulder as a point:
# ch=scipy.spatial.ConvexHull(np.append(gd['smooth']['RWrist'][:,0:3]-gd['smooth']['RShoulder'][:,0:3],[[0,0,0]],axis=0))

In [ ]:
forearm_length=np.sqrt(np.sum((gd['smooth']['RWrist'][:,:3]-gd['smooth']['RElbow'][:,:3])**2,axis=1))
upper_arm_length=np.sqrt(np.sum((gd['smooth']['RElbow'][:,:3]-gd['smooth']['RShoulder'][:,:3])**2,axis=1))
med_arm_length=np.median(forearm_length+upper_arm_length)

In [ ]:
norm_ch = ch.volume/(4/3*np.pi*med_arm_length**3)
norm_ch